In [27]:
import numpy as np
import torch
from models.cnn import CNN
from torchvision.transforms import Compose, ToTensor
from PIL import Image
from scipy.ndimage import binary_closing, binary_opening
from utils.display import show_image
from utils.data_augmentation import pad_tensor_with_borders

model = CNN()
model_name="modelb128e100_more_data_arch.pth"
model.load_state_dict(torch.load(f'models/{model_name}'))
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ground_truth = Image.open('data/sfa/ORI/img (92).jpg')
original_width, original_height = ground_truth.size

In [28]:
patch_size = 35
half_patch_size = patch_size // 2
stride = 1
padded_image = pad_tensor_with_borders(Compose([ToTensor()])(ground_truth), original_height + half_patch_size, original_width + half_patch_size)
show_image(padded_image)

In [29]:
def get_image_predictions(padded_image):
    
    unfolded = padded_image.unfold(1, patch_size, stride).unfold(2, patch_size, stride)
    unfolded = unfolded.permute(1, 2, 0, 3, 4).contiguous()
    patches = unfolded.view(-1, 3, patch_size, patch_size)

    num_patches = patches.size(0)
    batch_size = 1024
    num_batches = int(np.ceil(num_patches / batch_size))
    
    all_predictions = []
    with torch.no_grad():
        model.to(device)
        for i in range(num_batches):
            start_idx = i * batch_size
            end_idx = min(start_idx + batch_size, num_patches)
            batch_patches = patches[start_idx:end_idx]
            # Move the batch to the same device as the model
            batch_patches = batch_patches.to(device)
            # Predict using the trained model
            predictions = model(batch_patches)
            # Move predictions to CPU and convert to numpy
            predictions = predictions.cpu().numpy()
            
            all_predictions.append(predictions)
    
    # Concatenate all predictions into a single array
    all_predictions = np.concatenate(all_predictions, axis=0)
    
    return all_predictions.reshape(original_height, original_width)

In [30]:
def draw_mask(image_predictions, display=False):
    new_image = Image.new('L', (original_width, original_height), 0)
    # Iterate over each pixel in the new image and set it based on predictions
    for y in range(original_height):
        for x in range(original_width):
            prediction = image_predictions[y, x]
            new_pixel_value = 255 if prediction >= 0.5 else 0
            new_image.putpixel((x, y), new_pixel_value)
    
    if display:
        new_image.show("predicted_image.jpg")
        
    return new_image

In [31]:
preds = get_image_predictions(padded_image)

In [32]:
mask = draw_mask(preds, False)

In [33]:
# Step 3: Smoothen the mask using morphological operations
mask_np = np.array(mask)
kernel = np.ones((3, 3), np.uint8)
mask_np = binary_closing(mask_np, structure=kernel)
mask_np = binary_opening(mask_np, structure=kernel)
smoothened_mask = Image.fromarray((mask_np * 255).astype(np.uint8))

In [34]:
# Step 4: Extract the skin area
skin_area = Image.composite(ground_truth, Image.new('RGB', (original_width, original_height)), smoothened_mask)

In [35]:
# Step 5: Evaluation (Example, assuming ground truth is available)
ground_truth = Image.open('data/sfa/GT/img (92).jpg').convert('L')
ground_truth_np = np.array(ground_truth)
smoothened_mask_np = np.array(smoothened_mask)
ground_truth_np_skin = (ground_truth_np > 15).astype(int)

TP = np.sum((ground_truth_np_skin == 1) & (smoothened_mask_np == 255))
TN = np.sum((ground_truth_np_skin == 0) & (smoothened_mask_np == 0))
FP = np.sum((ground_truth_np_skin == 0) & (smoothened_mask_np == 255))
FN = np.sum((ground_truth_np_skin == 1) & (smoothened_mask_np == 0))

print(f"True Positives: {TP}")
print(f"True Negatives: {TN}")
print(f"False Positives: {FP}")
print(f"False Negatives: {FN}")

# Display results
ground_truth.show(title='Original Image')
smoothened_mask.show(title='Skin Mask')
skin_area.show(title='Extracted Skin Area')

True Positives: 38128
True Negatives: 330909
False Positives: 2703
False Negatives: 21476


In [36]:
TP, TN, FP, FN = map(float, [TP, TN, FP, FN])

accuracy = (TP + TN) / (TP + TN + FP + FN)
precision = TP / (TP + FP) if (TP + FP) != 0 else 0
recall = TP / (TP + FN) if (TP + FN) != 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

mcc_numerator = (TP * TN) - (FP * FN)
mcc_denominator = ((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN)) ** 0.5
mcc = mcc_numerator / mcc_denominator if mcc_denominator != 0 else 0

iou = TP / (TP + FP + FN)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")
print(f"MCC: {mcc:.4f}")
print(f"IoU: {iou:.4f}")

Accuracy: 0.9385
Precision: 0.9338
Recall: 0.6397
F1 Score: 0.7593
MCC: 0.7425
IoU: 0.6119


In [38]:
import csv

# Save metrics to CSV file
with open("results/evaluation_results.csv", 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([model_name])
    writer.writerow(['Accuracy', 'Precision', 'Recall', 'F1 Score', 'MCC', 'IoU'])
    writer.writerow([accuracy, precision, recall, f1_score, mcc, iou])